# Building Dataset 
We build a Pytorch Dataset Object to store and structure the graph data: 

In [2]:
# import libraries : 

import os 
import random 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 

# set random seed : 
np.random.seed( 41 )

import torch
import torch.nn as nn 
import torch.utils.data as data 
import torch.functional as F 
from torch import Tensor 

import torch_geometric
from torch_geometric.data import Data
# from typing import Optional, Tuple

In [ ]:
class EventData(data.Dataset): 
    
    # initialaize the event dataset 
    def __init__(self,path:str)->None:
        '''
        Inputs : 
            path: path to the folder where the csv file was contained.  
        '''
        super(EventData,self).__init__()
        self.events = [code[:-9] for code in os.listdir(path) if code.endswith('-hits.csv')]
        self.num_events = len(self.events)
        self.path = path 
        
    # function returns graph type represntation of the event dataset 
    def GraphData(self,idx:int) -> Data :
        eventid = self.events[idx] 
        
        hits = pd.read_csv(self.path+eventid+'-hits.csv')
        # total number of hits : these form the NODES of our graph. 
        nhits = hits.shape[0] 
        # x , y , z of the hits ! 
        hits_spatial = hits.to_numpy()[: , 1:4 ]
        # id related to the hits 
        hits_id = hits.to_numpy(dtype = int)[: ,[0,*list(range(4,7))]]

        particle = pd.read_csv(self.path+eventid+'-particles.csv')
        
        truth = pd.read_csv(self.path+eventid+'-truth.csv')
        
        